In [1]:
import os
import pytz
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from django.apps import apps as django_apps
from django.db.models.signals import post_save, post_delete
from edc_base.utils import get_utcnow
from edc_appointment.models import Appointment
from edc_appointment.constants import NEW_APPT
from edc_visit_schedule import site_visit_schedules

from copy import copy

In [2]:
subject_identifier = 'B142-040990044-5'
child_subject_identifier = 'B142-040990044-5-60'

In [3]:
def update_forms_visit(forms, visit=None, new_visit=None, form_type='crf'):
    for form in forms:
        kwargs = {}
        model_cls = django_apps.get_model(form.model)
        if form_type == 'requisition':
            kwargs = {'panel__name': form.panel_name}
            post_save.disconnect(dispatch_uid='senaite_sample_create_on_post_save')
        try:
            model_obj = model_cls.objects.get(maternal_visit=visit, **kwargs)
        except model_cls.DoesNotExist:
            continue
        else:
            print(model_obj)
            model_obj.maternal_visit = new_visit
            model_obj.save()

In [4]:
appt_list = ['appt_type', 'appt_status', 'appt_reason', 'comment', 'appt_datetime']

In [5]:
onsch = OnScheduleCohortCFUQuarterly.objects.get(
    subject_identifier=subject_identifier, child_subject_identifier=child_subject_identifier)
old_quart_appt = Appointment.objects.filter(
    subject_identifier=subject_identifier, schedule_name=onsch.schedule_name)
old_quart_appt.delete()
onsch.delete()

(1, {'flourish_caregiver.OnScheduleCohortCFUQuarterly': 1})

In [6]:
ssh = SubjectScheduleHistory.objects.filter(
    subject_identifier=subject_identifier, schedule_name=onsch.schedule_name)
ssh.delete()

(1, {'edc_visit_schedule.SubjectScheduleHistory': 1})

In [7]:
onschedule = OnScheduleCohortCFUSeq.objects.get(
    subject_identifier=subject_identifier, child_subject_identifier=child_subject_identifier)

try:
    csh = CohortSchedules.objects.get(
        schedule_type='sq_followup', cohort_name='cohort_c', child_count=1)
except CohortSchedules.DoesNotExist:
    print('meh')
else:
    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
        onschedule_model=csh.onschedule_model, name=csh.schedule_name)

    if not schedule.is_onschedule(
        subject_identifier=subject_identifier, report_datetime=onschedule.onschedule_datetime):
        base_appt_datetime = onschedule.onschedule_datetime

        schedule.put_on_schedule(
            subject_identifier=onschedule.subject_identifier, 
            onschedule_datetime=onschedule.onschedule_datetime,
            schedule_name=csh.schedule_name,
            base_appt_datetime=base_appt_datetime)

        onschedule_cls = django_apps.get_model(csh.onschedule_model)

        onsch = onschedule_cls.objects.get(
            subject_identifier=subject_identifier,
            schedule_name=csh.schedule_name)
        onsch.child_subject_identifier=child_subject_identifier
        onsch.onschedule_datetime = onschedule.onschedule_datetime
        onsch.save()

        complete_appts = Appointment.objects.filter(
            subject_identifier=subject_identifier,
            schedule_name=onschedule.schedule_name, ).exclude(appt_status=NEW_APPT)

        for appt in complete_appts:
            # Get corresponding appt
            new_appt = Appointment.objects.filter(subject_identifier=subject_identifier,
                                                  visit_code=appt.visit_code,
                                                  visit_code_sequence=appt.visit_code_sequence,
                                                  schedule_name=csh.schedule_name)
            if new_appt.exists():
                appt_dict = {key: getattr(appt, key) for key in appt_list}
                new_appt.update(**appt_dict)
                try:
                    visit = MaternalVisit.objects.get(appointment=appt)
                except MaternalVisit.DoesNotExist:
                    continue
                else:
                    new_visit = copy(visit)
                    new_visit.pk = None
                    new_visit.appointment = new_appt[0]
                    new_visit.schedule_name = new_appt[0].schedule_name
                    new_visit.visit_schedule_name = new_appt[0].visit_schedule_name
                    new_visit.save()

                    crfs = CrfMetadata.objects.filter(
                        subject_identifier=subject_identifier,
                        visit_code=visit.visit_code,
                        visit_code_sequence=visit.visit_code_sequence,
                        schedule_name=visit.schedule_name,
                        entry_status='KEYED')
                    print(visit, new_visit)
                    update_forms_visit(forms=crfs, visit=visit, new_visit=new_visit)

                    reqs = RequisitionMetadata.objects.filter(
                        subject_identifier=subject_identifier,
                        visit_code=visit.visit_code,
                        visit_code_sequence=visit.visit_code_sequence,
                        schedule_name=visit.schedule_name,
                        entry_status='KEYED')
                    update_forms_visit(forms=reqs, visit=visit, new_visit=new_visit, form_type='requisition')
                    visit.save()

B142-040990044-5 3000C.0 B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0
B142-040990044-5 3000C.0


In [10]:
post_delete.disconnect(dispatch_uid='metadata_reset_on_post_delete')

onsch = OnScheduleCohortCFUSeq.objects.get(
    subject_identifier=subject_identifier,
    child_subject_identifier=child_subject_identifier, schedule_name__icontains='2')
ssh = SubjectScheduleHistory.objects.filter(
    subject_identifier=subject_identifier, schedule_name=onsch.schedule_name)
appts = Appointment.objects.filter(
    subject_identifier=subject_identifier, schedule_name=onsch.schedule_name)
for appt in appts:
    visit = MaternalVisit.objects.get(appointment=appt)
    visit.delete()
appts.delete()
ssh.delete()
onsch.delete()

(1, {'flourish_caregiver.OnScheduleCohortCFUSeq': 1})